# Tutorial 7: Making data computer-readable and automate analysis

The way we write data on an excel sheet is rarely suitable for its use in large-scale analysis. We are living in the moment and write it all the way that **looks** more understandable. However, computers are too boring and don't care about visuals. They like strict formats so they can do what they do best: be fast dumbs. 

For that, we're going to start this tutorial away from this spreadsheet. We're going to convert a dataset that Roswita very generously provided for its use in computer. Don't worry, we're not going to do the whole boring task, just enough to understand what needs to be done and then we'll work with the already converted file `computer_readable_roswita_data.csv`. 

Pay attention to my screen. 

We have a computer-readable file! Perfect! Now we can do cool things with it. Let's start arranging the data with our old friend `pandas`. 

In [ ]:
import pandas as pd

Import the csv file

In [ ]:
df = pd.read_csv('computer_readable_roswita_data.csv')

We can see in the dataframe that there are two kinds of entries: blank and measurements. Blank has only one value, which is the value to substract from the other measurements to normalize our absorbance read.

In [ ]:
df

Let's select only the entries in the dataframe that are measurements and stored it in another dataframe. You know I get annoyed at indeces that start at 1, so I'm going to reset it and have it at 0 again. 

In [ ]:
samples_df = df[df['Type'] == 'measurement'].reset_index(drop=True)
samples_df

## Normalize the measurements

We need to select the blank, but if we select it indicating the column that we want, in the pandas way, we still get a series, **not yet a float**. 

In [ ]:
print(df[df['Type']=='blank']['raw_absorbance'])
print(type(df[df['Type']=='blank']['raw_absorbance']))

Since the lenght of the series is 1, we can select element 0 and obtain the float. 

In [ ]:
blank = df[df['Type']=='blank']['raw_absorbance'][0]
print(blank)
print(type(blank))

### Create the normalized column

In [ ]:
samples_df['normalized_absorbance'] = samples_df['raw_absorbance']-blank
samples_df

I also want to convert some float values into integers, since I'm treating this data more as indices, more as a personal thing than an actual requirement, so I can do this with this for loop. 

In [ ]:
to_int_list = ['condition', 'replicate', 'outlier', 'time']  # Define which columns I want to become integers
for column_name in to_int_list:
    samples_df[column_name] = samples_df[column_name].astype('int')  # Convert each column to integer

In [ ]:
samples_df

## Plot the data

There're several libraries on python for plotting. We have already seen matplotlib, which we're still going to use in this practical. Additionally, I want to give you a hint of a library called seaborn. I'm still a newbie with seaborn, and I often find myself unable to plot what I want with it. Nevertheless, its simplicity for compatibly-structured dataframes is ecstatic <3. 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
for condition in samples_df['condition'].unique():
    # The line below is all it's needed to plot, the rest is all aesthetics for the title
    ax = sns.scatterplot(data=samples_df[samples_df['condition'] == condition], x='time', y='normalized_absorbance', hue='replicate', palette=sns.color_palette(n_colors=3))
    
    tet = samples_df[samples_df['condition'] == condition]['ug/ml tet'].unique()[0]
    iptg = samples_df[samples_df['condition'] == condition]['mM IPTG'].unique()[0]
    plt.title('Experiment {0}: {1} ug/ml tet, {2} mM IPTG'.format(condition, tet, iptg))
    plt.show()

This is very pretty and easy, and seaborn is made to fit most data science requirements. However, I don't know how to do on seaborn (or if it's even possible) an exponential fit of the average values of 3 technical replictates on a log scale y-axis... what a mouthful!  

For that, we're going to implement them on matplotlib.pyplot. 

### Log-scaled y-axis and exponential fit (for linear growth detection)

**DON'T PANIC!**

Let's go throught these lines of code together: 

In [ ]:
import numpy as np

In [ ]:
for condition in samples_df['condition'].unique():
    plt.yscale("log", basey=10)  # set the y-axis to log10 scale
    
    # Select the times for each sample. In this case they are equal because we haven't thrown away any outlier. 
    time0 = samples_df[(samples_df['condition']==condition) & (samples_df['replicate']==0)]['time']
    time1 = samples_df[(samples_df['condition']==condition) & (samples_df['replicate']==1)]['time']
    time2 = samples_df[(samples_df['condition']==condition) & (samples_df['replicate']==2)]['time']
    
    absorbance0 = samples_df[(samples_df['condition']==condition) & (samples_df['replicate']==0)]['normalized_absorbance']
    absorbance1 = samples_df[(samples_df['condition']==condition) & (samples_df['replicate']==1)]['normalized_absorbance']
    absorbance2 = samples_df[(samples_df['condition']==condition) & (samples_df['replicate']==2)]['normalized_absorbance']
    
    ax0 = plt.scatter(time0, absorbance0, label = 'replicate 0')
    ax1 = plt.scatter(time1, absorbance1, label = 'replicate 1')
    ax2 = plt.scatter(time2, absorbance2, label = 'replicate 2')
    
    # This line wouldn't work if we remove a value unless we inlude it as a NaN (not a number, np.nan). 
    average = np.mean(np.array([absorbance0, absorbance1, absorbance2]), axis=0)
    
    plt.plot(time0, average, label='average', color='hotpink')  # Plot average of replicates
    
    exponent, base = np.polyfit(time0, np.log(average), 1)  # Exponential fit through the average values. 
    plt.plot(time0, np.exp(exponent*np.array(time0)) * np.exp(base), label = 'Exponential fit')  # Calculate fitted points for the times in time0
    plt.text(7,0.05,'Exponential fit\nBase:{0}\nExponent:{1}\n\n{0}e({1}x)'.format(round(np.exp(base),4),round(exponent,4)))  # Simply print the values for of the exponential fit
    
    # Diverse plot esthetics
    plt.legend()
    plt.xlabel('time')
    plt.ylabel('normalized_absorbance')
    plt.ylim(samples_df[samples_df['condition']==condition]['normalized_absorbance'].min(),samples_df[samples_df['condition']==condition]['normalized_absorbance'].max() )
    tet = samples_df[samples_df['condition'] == condition]['ug/ml tet'].unique()[0]
    iptg = samples_df[samples_df['condition'] == condition]['mM IPTG'].unique()[0]
    plt.title('Experiment {0}: {1} ug/ml tet, {2} mM IPTG'.format(condition, tet, iptg))
    plt.show()

There fits are not really that good, because of several reasons:
- From time 0 to time 1 hours, the cell haven't reached exponential growth yet
- Cells stop their exponential growth before the 11 hours. 

Because of that, the code has been modified to make the same calculation from time=1 until the hour we decide (note the +1, because of python's way of indexing). 

In [ ]:
exponential_stops_list = [4, 10, 4, 9]

for i, condition in enumerate(samples_df['condition'].unique()):
    plt.yscale("log", basey=10)

    raw_or_normalized = 'normalized_absorbance'
    
    print(exponential_stops_list[i])
    
    time0 = samples_df[(samples_df['condition']==condition) & (samples_df['replicate']==0)]['time'][1:exponential_stops_list[i]+1]
    time1 = samples_df[(samples_df['condition']==condition) & (samples_df['replicate']==1)]['time'][1:exponential_stops_list[i]+1]
    time2 = samples_df[(samples_df['condition']==condition) & (samples_df['replicate']==2)]['time'][1:exponential_stops_list[i]+1]
    
    absorbance0 = samples_df[(samples_df['condition']==condition) & (samples_df['replicate']==0)][raw_or_normalized][1:exponential_stops_list[i]+1]
    absorbance1 = samples_df[(samples_df['condition']==condition) & (samples_df['replicate']==1)][raw_or_normalized][1:exponential_stops_list[i]+1]
    absorbance2 = samples_df[(samples_df['condition']==condition) & (samples_df['replicate']==2)][raw_or_normalized][1:exponential_stops_list[i]+1]
    
    ax0 = plt.scatter(time0, absorbance0, label = 'replicate 0')
    ax1 = plt.scatter(time1, absorbance1, label = 'replicate 1')
    ax2 = plt.scatter(time2, absorbance2, label = 'replicate 2')
    
    average = np.mean(np.array([absorbance0, absorbance1, absorbance2]), axis=0)
    
    plt.plot(time0, average, label='average', color='hotpink')  # Plot average of replicates

    exponent, base = np.polyfit(time0, np.log(average), 1)
    plt.plot(time0, np.exp(exponent*np.array(time0)) * np.exp(base), label = 'Exponential fit')
    plt.text(exponential_stops_list[i]-2,0.05,'Exponential fit\nBase:{0}\nExponent:{1}\n\n{0}e({1}x)'.format(round(np.exp(base),3),round(exponent,3)))
    
    plt.legend()
    plt.xlabel('time')
    plt.ylabel(raw_or_normalized)
    plt.ylim(samples_df[samples_df['condition']==condition][raw_or_normalized].min(),samples_df[samples_df['condition']==condition][raw_or_normalized].max() )
    tet = samples_df[samples_df['condition'] == condition]['ug/ml tet'].unique()[0]
    iptg = samples_df[samples_df['condition'] == condition]['mM IPTG'].unique()[0]
    plt.title('Experiment {0}: {1} ug/ml tet, {2} mM IPTG'.format(condition, tet, iptg))
    plt.show()

## Homework

- Make one of your datasets computer-readable, read it with pandas and make a cute plot (and fit if applicable) :)
- (optional, potentially tricky) Remove the outliers from experiments 1 and 3 and redo the fit and plotting